In [1]:
import pandas as pd 

import pyspark

from pyspark.ml.classification import LogisticRegression

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml import Pipeline


from pyspark.ml.feature import StringIndexer, VectorIndexer, StringIndexerModel, IndexToString
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [17]:
train = spark.read.format("csv").load("C:\\Users\\Asus\\Documents\\HK6\\CS338\\creditcard\\kafka_producer_consumer\\creditcard.csv",header = 'True',inferSchema='True')

In [3]:
train.show(5)

+----+------------------+-------------------+----------------+------------------+-------------------+-------------------+-------------------+------------------+------------------+-------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-------------------+------------------+-------------------+--------------------+-------------------+------------------+------------------+------------------+------------------+--------------------+-------------------+------+-----+
|Time|                V1|                 V2|              V3|                V4|                 V5|                 V6|                 V7|                V8|                V9|                V10|               V11|               V12|               V13|               V14|               V15|               V16|               V17|                V18|               V19|                V20|                 V21|                V22|     

In [19]:
train_data, test_data = train.randomSplit([0.8, 0.2], seed=42)
train_class =train_data.filter(train_data.Class == 1).count()
test_class = test_data.filter(test_data.Class == 1).count()
print(train_class,test_class)

400 92


In [7]:
from xgboost.spark import SparkXGBClassifier

spark_reg_estimator = SparkXGBClassifier(
  features_col="features",
  label_col="Class",
  num_workers=2,
)

In [10]:
from pyspark.ml.feature import RobustScaler
scaler = RobustScaler(inputCol="features_T_A", outputCol="scaledFeatures")
assembler = VectorAssembler().setInputCols(['Time','Amount']).setOutputCol("features_T_A")
assembler_2 = VectorAssembler().setInputCols(['V1','V2','V3','V4','V5','V6','V7','V8','V9','V10','V11','V12','V13','V14','V15','V16','V17','V18','V19','V20','V21','V22','V23','V24','V25','V26','V27','V28',"scaledFeatures"]).setOutputCol("features")
pipeline = Pipeline( stages = [assembler,scaler,assembler_2,spark_reg_estimator])

In [ ]:
df_train = pipeline.fit(train_data).transform(train_data)
df_train.show(5)

In [ ]:
model = pipeline.fit(train_data)

In [5]:
assembler = VectorAssembler(inputCols=['Time','Amount'],outputCol="features")
df_train = assembler.transform(train)
df_train.show(5)

+----+------+-----+------------+
|Time|Amount|Class|    features|
+----+------+-----+------------+
| 0.0|149.62|    0|[0.0,149.62]|
| 0.0|  2.69|    0|  [0.0,2.69]|
| 1.0|378.66|    0|[1.0,378.66]|
| 1.0| 123.5|    0| [1.0,123.5]|
| 2.0| 69.99|    0| [2.0,69.99]|
+----+------+-----+------------+
only showing top 5 rows



In [6]:
lgr = LogisticRegression(labelCol='Class',featuresCol='features')
lgr.fit(df_train).transform(df_train).show(5)

+----+------+-----+------------+--------------------+--------------------+----------+
|Time|Amount|Class|    features|       rawPrediction|         probability|prediction|
+----+------+-----+------------+--------------------+--------------------+----------+
| 0.0|149.62|    0|[0.0,149.62]|[5.79307872428651...|[0.99696068363489...|       0.0|
| 0.0|  2.69|    0|  [0.0,2.69]|[5.82337750633479...|[0.99705112264943...|       0.0|
| 1.0|378.66|    0|[1.0,378.66]|[5.74585418263933...|[0.99681417848171...|       0.0|
| 1.0| 123.5|    0| [1.0,123.5]|[5.79847132840560...|[0.99697697993881...|       0.0|
| 2.0| 69.99|    0| [2.0,69.99]|[5.80951208906387...|[0.99701007356111...|       0.0|
+----+------+-----+------------+--------------------+--------------------+----------+
only showing top 5 rows



In [7]:
pipeline = Pipeline( stages = [assembler,lgr, ])
model = pipeline.fit(train)

In [8]:
model.write().overwrite().save("./model")

In [20]:
from pyspark.ml import PipelineModel
model = PipelineModel.read().load("./model")

In [27]:
result = model.transform(train_data)


In [22]:
result.show(5)

+----+------------------+-----------------+------------------+------------------+------------------+------------------+--------------------+-------------------+------------------+-------------------+------------------+------------------+-------------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+------------------+-------------------+------------------+-------------------+------------------+------------------+-------------------+-------------------+-------------------+------+-----+------------+--------------------+--------------------+--------------------+----------+--------------------+
|Time|                V1|               V2|                V3|                V4|                V5|                V6|                  V7|                 V8|                V9|                V10|               V11|               V12|                V13|               V14|                V15|                V16|           

In [28]:
y_pred = result.select('prediction').collect()
y_pred_array = [x[0] for x in y_pred]
# y_pred_array
y = train_data.select('Class').collect()
y_array = [x[0] for x in y]
# y_array

In [29]:
from sklearn.metrics import classification_report
print(classification_report(y_array, y_pred_array))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00    227645
           1       0.99      0.96      0.98       400

    accuracy                           1.00    228045
   macro avg       1.00      0.98      0.99    228045
weighted avg       1.00      1.00      1.00    228045



In [31]:
count = 0
for i in y_pred_array:
    if i == 1:
        count += 1
count

387

In [3]:
print("Spark version = {}".format(spark.sparkContext.version))
print(f"Hadoop version = {spark._sc._jvm.org.apache.hadoop.util.VersionInfo.getVersion()}")

Spark version = 3.5.1
Hadoop version = 3.3.4
